In [95]:
import nltk
from nltk.corpus import brown
from nltk.data import find
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
from gensim.models import Word2Vec

In [96]:
df = pd.read_csv('data/NN_Data.csv').drop(columns=['Unnamed: 0'])
df["no_punct"] = df['text'].str.replace('[^\w\s]','')
df['str_list'] = df.no_punct.apply(lambda s: s.split(' '))

In [97]:
m = Word2Vec(df['str_list'].tolist(), min_count=1, vector_size = 300,workers=3, window =3, sg = 1)

In [98]:
word_vectors = m.wv

In [99]:
word_vectors.save('vectors.kv')
model = gensim.models.KeyedVectors.load('vectors.kv')

In [100]:
word_vectors

In [86]:
#len(m.wv)

model[0]
-0.12155986577272415

array([-1.21559866e-01,  3.10608387e-01, -7.96706006e-02,  2.87860394e-01,
       -4.36797142e-01, -2.14242652e-01,  3.16138923e-01,  3.97187501e-01,
        3.93480249e-02, -1.44469693e-01,  2.98315048e-01, -3.73451263e-02,
       -1.29558563e-01,  2.43326779e-02, -2.64992863e-01, -6.85425475e-02,
        2.84688771e-01,  3.80410366e-02,  5.98975597e-03, -1.43221721e-01,
        8.87219608e-02,  1.34182721e-01,  9.82196182e-02,  1.82634547e-01,
        2.53480732e-01, -1.19472854e-01, -3.41972858e-01, -1.60899043e-01,
       -4.90151137e-01, -3.60178500e-01,  8.71084072e-03, -3.99327688e-02,
        2.75830775e-01, -3.07274669e-01, -2.17069030e-01,  4.45033088e-02,
        1.30309686e-01, -2.01842129e-01, -1.20238073e-01, -2.50188380e-01,
       -3.94649565e-01, -6.53673336e-02, -3.82278293e-01, -1.19391784e-01,
        1.85716301e-01,  1.81374446e-01, -4.78640720e-02,  3.35687518e-01,
        4.04683411e-01,  1.37748465e-01,  1.71608493e-01,  1.46581270e-02,
       -3.60404551e-02,  

In [101]:
EMBEDDING_DIM = len(model['heart'])      # we know... it's 300

# initialize embedding matrix and word-to-id map:
embedding_matrix = np.zeros((len(model) + 1, EMBEDDING_DIM))       
vocab_dict = {}

# build the embedding matrix and the word-to-id map:
for i, word in enumerate(model.keys()):
    embedding_vector = model.key_to_index[word]
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        vocab_dict[word] = i

AttributeError: 'KeyedVectors' object has no attribute 'keys'

In [102]:
!pip show gensim

Name: gensim
Version: 4.0.1
Summary: Python framework for fast Vector Space Modelling
Home-page: http://radimrehurek.com/gensim
Author: Radim Rehurek
Author-email: me@radimrehurek.com
License: LGPL-2.1-only
Location: /Users/swethapola/opt/anaconda3/lib/python3.8/site-packages
Requires: scipy, smart-open, numpy
Required-by: 


In [103]:
model.keys()

AttributeError: 'KeyedVectors' object has no attribute 'keys'

In [114]:
words = list(m.wv.vocab)


AttributeError: The vocab attribute was removed from KeyedVector in Gensim 4.0.0.
Use KeyedVector's .key_to_index dict, .index_to_key list, and methods .get_vecattr(key, attr) and .set_vecattr(key, attr, new_val) instead.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4